# Entrenamiento del modelo LeNet-5 con el dataset MNIST

Esta libreta cuenta con toda la funcionalidad básica para descargar un dataset de TorchVision, crear un modelo sobre PyTorch desde cero y empezar a entrenar. Hay huecos en blanco señalizados con #[!] al final y enlaces a la documentación de PyTorch para que exploréis distintas opciones.
#### Opciones a explorar
**Función de activación:** https://pytorch.org/docs/stable/nn#non-linear-activations-weighted-sum-nonlinearity \
&nbsp;&nbsp;&nbsp;&nbsp;Opciones: Sigmoid, Tanh y ReLU \
&nbsp;&nbsp;&nbsp;&nbsp;Ejemplo: self.activacion1 = nn.Sigmoid()

**Capa de Pooling:** https://pytorch.org/docs/stable/nn#pooling-layers \
&nbsp;&nbsp;&nbsp;&nbsp;Opciones: MaxPool2d y AvgPool2d \
&nbsp;&nbsp;&nbsp;&nbsp;(Utilizar kernel_size=2, stride=2) \
&nbsp;&nbsp;&nbsp;&nbsp;Ejemplo: self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)

**Optimizador:** https://pytorch.org/docs/stable/optim \
&nbsp;&nbsp;&nbsp;&nbsp;Opciones: SGD y Adam \
&nbsp;&nbsp;&nbsp;&nbsp;Hiperparámetros interesantes: learning rate, weight_decay \
&nbsp;&nbsp;&nbsp;&nbsp;Ejemplo: optimizer = optim.SGD(model.parameters(), lr=0.01)


In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

import numpy as np
import matplotlib.pyplot as plt

## Conjuntos de datos
En el siguiente enlace teneis todos los datasets disponibles en PyTorch para el reconocimiento de imágenes.\
https://pytorch.org/vision/0.20/datasets

Dependiendo del dataset, el preprocesado de las muestras puede variar mucho. La mayoria requirere normalizar los datos.
En la propia función de transformación que se utiliza para el preprocesado, se puede incluir aumento de datos.\
https://pytorch.org/vision/stable/transforms \
https://pytorch.org/vision/0.13/transforms

In [ ]:
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
# Normalizar
train_set = torch.stack([img for img, _ in train_set], dim=0)
mean = train_set.mean()
std = train_set.std()

batch_size = 16 # Tamaño de los lotes de muestras
# Preprocesado de las muestras
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=mean, std=std)])

# Carga el dataset de entrenamiento y validación ya normalizado
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)

#### Mostrar ejemplos de imágenes del conjunto de datos

In [ ]:
fig = plt.figure()
label_names = train_set.classes
for data, labels in train_loader:
  data = data.permute(0, 2, 3, 1)
  h, w, c = data[0].shape
  print(f"Dimensiones de las imagenes - altura: {h}, anchura: {w}, canales: {c}")
  for i in range(7):
    plt.subplot(1,7,i+1)
    fig.tight_layout()
    plt.title(label_names[labels[i]])
    plt.imshow(data[i])
    plt.xticks([])
    plt.yticks([])
  break

## Modelo

#### LeNet-5
Entrada: 32x32 (imagen en escala de grises) \
**Extracción de carácteristicas** \
**Bloque 1:** \
&nbsp;&nbsp;&nbsp;&nbsp;**Conv 1:** 6 filtros 5×5, padding=2, stride=1 \
&nbsp;&nbsp;&nbsp;&nbsp;**Activación**  \
&nbsp;&nbsp;&nbsp;&nbsp;**Pool 1:** Tamaño de ventana 2×2, stride=2

**Bloque 2:** \
&nbsp;&nbsp;&nbsp;&nbsp;**Conv 2:** 16 filtros 5×5, sin padding, stride=1 \
&nbsp;&nbsp;&nbsp;&nbsp;**Activación**  \
&nbsp;&nbsp;&nbsp;&nbsp;**Pool 2:** Tamaño de ventana 2×2, stride=2

**Flatten** (aplana la imagen)

**Clasificador** \
&nbsp;&nbsp;&nbsp;&nbsp;**FC 1:** 120 neuronas \
&nbsp;&nbsp;&nbsp;&nbsp;**Activación**  \
&nbsp;&nbsp;&nbsp;&nbsp;**FC 2:** 84 neuronas \
&nbsp;&nbsp;&nbsp;&nbsp;**Activación**  \
&nbsp;&nbsp;&nbsp;&nbsp;**FC 3:** 10 neuronas \
&nbsp;&nbsp;&nbsp;&nbsp;**Softmax** activación (Ya implícita en la función de pérdida)

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        # Extracción de características
        # Bloque 1
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.activacion1 = ...                                                    # [!]
        self.pool1 = ...                                                          # [!]
        # Bloque 2
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.activacion2 = ...                                                    # [!]
        self.pool2 = ...                                                          # [!]

        # Clasificador
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.activacion3 = ...                                                    # [!]
        self.fc2 = nn.Linear(120, 84)
        self.activacion4 = ...                                                    # [!]
        self.fc3 = nn.Linear(84, 10)
        # self.softmax = nn.Softmax() # Ya implícita en la función de pérdida

    def forward(self, x):
        # Extracción de características
        # Bloque 1
        x = self.conv1(x)
        x = self.activacion1(x)
        x = self.pool1(x)
        # Bloque 2
        x = self.conv2(x)
        x = self.activacion2(x)
        x = self.pool2(x)

        # Flatten the tensor for fully connected layers
        x = x.view(-1, 16 * 5 * 5)

        # Clasificador
        x = self.fc1(x)
        x = self.activacion3(x)
        x = self.fc2(x)
        x = self.activacion4(x)
        x = self.fc3(x) 
        # x = self.softmax(x) # Ya implícita en la función de pérdida

        return x

## Entrenamiento
**Inicializamos:** \
1- Modelo \
2- Función de pérdida \
3- Optimizador

**Cada época en entrenamiento:** \
&nbsp;&nbsp;&nbsp;&nbsp;Cargamos imágenes y etiquetas en dispositivo \
&nbsp;&nbsp;&nbsp;&nbsp;Ponemos los gradientes a 0 \
&nbsp;&nbsp;&nbsp;&nbsp;Forward-Pass, obtenemos las predicciones \
&nbsp;&nbsp;&nbsp;&nbsp;Función de pérdida, comparando predicciones con etiquetas \
&nbsp;&nbsp;&nbsp;&nbsp;Backward-pass, se calculan los gradientes \
&nbsp;&nbsp;&nbsp;&nbsp;Optimizador, se actualizan los pesos

**Cada época en validación:** \
&nbsp;&nbsp;&nbsp;&nbsp;Cargamos imágenes y etiquetas en dispositivo \
&nbsp;&nbsp;&nbsp;&nbsp;Forward-Pass, obtenemos las predicciones \
&nbsp;&nbsp;&nbsp;&nbsp;Función de pérdida, comparando predicciones con etiquetas

In [ ]:
model = LeNet()                    # Inicialización
criterion = nn.CrossEntropyLoss()  # Función de pérdida
optimizer = ...                                                                            # [!]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Dispositivo
model.to(device)

n_epochs = 5 # Número de epocas a entrenar
# Almacenamos métricas
train_accuracies, val_accuracies = [], []
train_losses, val_losses = [], []
for epoch in range(n_epochs): 
    # Métricas
    running_loss, correctas, total = 0.0, 0, 0
    
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)            
        optimizer.zero_grad() # Reinicia el optimizador
        output = model(data) # Forward-Pass
        loss = criterion(output, target)
        loss.backward() # Backward-Pass
        optimizer.step() # Actualiza los pesos

        # Métricas
        running_loss += loss.item()
        _, prediccion = output.max(1)
        total += target.size(0)
        correctas += prediccion.eq(target).sum().item()
    train_loss = running_loss / len(train_loader)
    train_accuracy = 100. * correctas / total
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Validación del modelo
    # Métricas
    val_loss, correctas, total = 0.0, 0, 0
    
    model.eval()
    with torch.no_grad(): # No calculamos gradientes en validación
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)

            output = model(data) # Forward-Pass
            loss = criterion(output, target)

            # Métricas
            val_loss += loss.item()
            _, prediccion = output.max(1)
            total += target.size(0)
            correctas += prediccion.eq(target).sum().item()
    val_loss /= len(val_loader)
    val_accuracy = 100. * correctas / total
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    # Imprimimos las métricas por época
    print(f"Epoch [{epoch + 1}/{n_epochs}]", end=" | ")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%", end=" | ")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

In [ ]:
# Representamos la evolución del accuracy
plt.figure(figsize=(10, 5))
plt.plot(range(1, n_epochs + 1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, n_epochs + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy Evolution')
plt.legend()
plt.grid()
plt.show()

## Guardar y cargar un modelo entrenado

In [ ]:
def save_model(model, optimizer, path='./model.pth'):
    state = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }
    torch.save(state, path)
    print(f"Model saved to {path}")

def load_model(model, optimizer, path='./model.pth'):
    checkpoint = torch.load(path, weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'], strict=False)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    print(f"Model loaded from {path}, starting from epoch {epoch}")

In [ ]:
model_path = './model.pth'
# Guardar modelo
save_model(model, optimizer, path=model_path)

# Cargar modelo
model = LeNet()
optimizer = ... 
load_model(model, optimizer, path=model_path)